In [1]:
import nltk
import csv
nltk.download('punkt')


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [2]:
# move vocab out of the dataset class
# use train.csv for generating vicab
# divide train into train, val and test
# get dataloaders for each
import re
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader
import torch
def clean(st):
  s=st.lower()
  s=re.sub("\s(I'm)\s"," I am ",s)
  s=re.sub("\s(don't)\s"," do not ",s)
  s=re.sub("\s(can't)\s"," can not ",s)
  s=re.sub("\s(doesn't)\s"," do not ",s)
  s=re.sub("\s(won't)\s"," will not ",s)
  s=re.sub("\s(it's)\s"," it is ",s)
  s=re.sub("[:`\"]","",s)
  s=re.sub("'ve"," have",s)
  s=re.sub("\.",". ",s)
  return s
def build_vocab(path):
  # creating vocab
  # create train data [[[tok1,tok2,tok3..],[op1,op2,op3....]],[[],[]]..]
  punct=[".",",","?","None"]
  vocab=[]
  data=[]
  num_rows=0
  with open(path, 'r') as f:
    csvfile=csv.reader(f)
    for line in csvfile:
      num_rows+=1
      if num_rows>1:
        if line[1]!="":
          line[1]=clean(line[1])
          data.append(line[1])
          vocab+=nltk.word_tokenize(line[1].lower())

  # print(len(set(vocab)))
  vocab=set(vocab)
  vocab={word:i for i,word in enumerate(vocab)}
  l=len(vocab)
  vocab['OOV']=l
  vocab["<EOS>"]=l+1
  vocab["<BOS>"]=l+2
  vocab["<PAD>"]=l+3
  idx_to_word={i:word for word,i in vocab.items()}

  punct={p:i for i,p in enumerate(punct)}
  idx_to_punct={i:p for p,i in punct.items()}
  maxi_len=0

  print(len(data))
  return vocab,punct,idx_to_word,idx_to_punct

# dividing into train, test, val data
def dividedata(path,vocab,punct,idx_to_word,idx_to_punct):

  data=[]
  num_rows=0
  with open(path, 'r') as f:
    csvfile=csv.reader(f)
    for line in csvfile:
      num_rows+=1
      if num_rows>1:
        if line[1]!="":
          line[1]=clean(line[1])
          data.append(line[1])
    tot_data=[]
    maxi_len=0

    for j in range(len(data)):
      words=nltk.word_tokenize(data[j])
      # print("words len ",len(words))
      inp=[vocab["<BOS>"]]
      non="None"
      op=[punct[non]]
      for i in range(len(words)):
        if words[i] not in punct:
          inp.append(vocab.get(words[i],vocab['OOV']))
          if i+1<len(words) and words[i+1] in punct:
            op.append(punct[words[i+1]])
            continue
          else:
            op.append(punct[non])
        elif words[i]==".":
          inp.append(vocab["<EOS>"])
          op.append(punct[non])
          if i<len(words)-1:
            inp.append(vocab["<BOS>"])
            op.append(punct[non])
      if len(inp)==len(op):
        tot_data.append([inp,op])
        maxi_len=max(maxi_len,len(inp))
      else:
        print("err")
    print("Number of samples : ",len(tot_data))
    print("Each sample : ",len(tot_data[0]))
    X=[x[0] for x in tot_data]
    y=[y[1] for y in tot_data]



    # print(len(X))
    X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)
    X_val, X_test, y_val, y_test = train_test_split(X_val, y_val, test_size=0.3, random_state=42)

    print([idx_to_word[x] for x in X_val[0]])
    print([idx_to_punct[x] for x in y_val[0]])
    return X_train,y_train,X_val,y_val,X_test,y_test


vocab,punct,idx_to_word,idx_to_punct=build_vocab('train.csv')
X_train,y_train,X_val,y_val,X_test,y_test=dividedata('train.csv',vocab,punct,idx_to_word,idx_to_punct)


3508
Number of samples :  3508
Each sample :  2
['<BOS>', 'hi', '<EOS>', '<BOS>', 'good', 'for', 'you', 'in', 'planning', 'ahead', 'to', 'do', 'what', "'s", 'healthiest', 'for', 'your', 'baby', '(', 'and', 'yourself', ')', '<EOS>', '<BOS>', 'that', "'s", 'a', 'great', 'first', 'step', '!', 'it', 'is', 'also', 'good', 'that', 'you', 'are', 'able', 'to', 'identify', 'that', 'it', 'is', 'not', 'always', 'a', 'physical', 'need', 'that', "'s", 'driving', 'the', 'addiction', '<EOS>', '<BOS>', 'for', 'the', 'next', 'steps', 'i', 'would', 'suggest', 'trying', 'to', 'figure', 'out', 'when', 'the', 'psychological', 'cravings', 'for', 'a', 'cigarette', 'occur', '<EOS>', '<BOS>', 'the', 'psychological', '(', 'or', 'mental', ')', 'cravings', 'are', 'usually', 'based', 'out', 'of', 'habit', 'such', 'as', 'having', 'a', 'cigarette', 'after', 'a', 'meal', '<EOS>', '<BOS>', 'and', 'if', 'you', "'re", 'consciously', 'trying', 'to', 'quit', 'you', "'ll", 'find', 'the', 'craving', 'starts', 'with', 'simpl

In [46]:
print(idx_to_punct)

{0: '.', 1: ',', 2: '?', 3: 'None'}


In [ ]:
# re-checking
# print([idx_to_word[x] for x in X_val[0]])
# print([idx_to_punct[x] for x in y_val[0]])

['<BOS>', 'hi', '<EOS>', '<BOS>', 'good', 'for', 'you', 'in', 'planning', 'ahead', 'to', 'do', 'what', "'s", 'healthiest', 'for', 'your', 'baby', '(', 'and', 'yourself', ')', '<EOS>', '<BOS>', 'that', "'s", 'a', 'great', 'first', 'step', '!', 'it', 'is', 'also', 'good', 'that', 'you', 'are', 'able', 'to', 'identify', 'that', 'it', 'is', 'not', 'always', 'a', 'physical', 'need', 'that', "'s", 'driving', 'the', 'addiction', '<EOS>', '<BOS>', 'for', 'the', 'next', 'steps', 'i', 'would', 'suggest', 'trying', 'to', 'figure', 'out', 'when', 'the', 'psychological', 'cravings', 'for', 'a', 'cigarette', 'occur', '<EOS>', '<BOS>', 'the', 'psychological', '(', 'or', 'mental', ')', 'cravings', 'are', 'usually', 'based', 'out', 'of', 'habit', 'such', 'as', 'having', 'a', 'cigarette', 'after', 'a', 'meal', '<EOS>', '<BOS>', 'and', 'if', 'you', "'re", 'consciously', 'trying', 'to', 'quit', 'you', "'ll", 'find', 'the', 'craving', 'starts', 'with', 'simply', 'thinking', 'about', 'having', 'a', 'cigaret

In [3]:
import csv
import re
import nltk
from torch.utils.data import Dataset
from torch.nn.utils.rnn import pad_sequence

class dataset(Dataset):
  x=[]
  y=[]

  def __init__(self,x,y):
    self.x=x
    self.y=y

  def __len__(self):
      return len(self.x)

  def __getitem__(self,index:int):
    try:
      _=self.x[index]
      return [self.x[index],self.y[index]]
    except:
      print(index)


train_obj=dataset(X_train,y_train)
val_obj=dataset(X_val,y_val)
test_obj=dataset(X_test,y_test)
print(len(train_obj.x))
print(len(val_obj.x))
print(len(test_obj.x))


2806
491
211


In [4]:
# issues with vocab
# 1. words joined by . (hope.however, died.then)- split using period
# 2. numbers like 501, 404
# 3. phone numbers
# 4. urls
print(len(vocab))
print(vocab)
print(train_obj.x[0])
print(train_obj.y[0])

12452
{'vent': 0, 'serenity': 1, 'cyst': 2, 'awareness': 3, 'deseado': 4, 'org/tests-procedures/light-therapy/home/ovc-20197416recently': 5, 'principle': 6, 'hey': 7, 'esther': 8, 'delusions': 9, 'pleasures': 10, 'pastoral': 11, 'toy': 12, 'plan': 13, 'distorting': 14, 'prompting': 15, 'guides': 16, 'other-half': 17, 'com/statistics/http//fathers': 18, 'domino': 19, 'eachother': 20, 'puedas': 21, 'embarrassing': 22, 'therapybyshannon': 23, '800-273-8255': 24, 'break': 25, 'manages': 26, 'infedienity': 27, 'self-examine': 28, 'caveat': 29, 'mama': 30, 'hang': 31, 'absorbs': 32, 'geared': 33, 'incluye': 34, '[': 35, 'or/your': 36, 'wore': 37, 'cayendo': 38, 'intermitente': 39, 'war': 40, 'healed': 41, 'sunlightget': 42, 'ball': 43, 'lyrics': 44, 'engines': 45, 'numbersthe': 46, 'clash': 47, 'irreducible': 48, 'intimidating': 49, 'strains': 50, 'incontinent': 51, 'hasta': 52, 'insults': 53, 'strategize': 54, 'landed': 55, 'balance': 56, '787-466-5478': 57, 'worship': 58, 'cross-dresser': 

In [5]:
# divide data into train, val, and test

pad_idx=vocab['<PAD>']
pad_idx_punct=punct["None"]
def custom_collate(batch):
  print("Length of batch ",len(batch), len(batch[0]))
  inputs=[torch.tensor(elem[0]) for elem in batch]
  lengths=[len(elem[0]) for elem in batch]
  print("len of inputs : ",len(inputs), len(inputs[0]))
  labels=[torch.tensor(elem[1]) for elem in batch]
  inputs=pad_sequence(inputs,batch_first=True,padding_value=pad_idx)
  labels=pad_sequence(labels,batch_first=True,padding_value=pad_idx_punct)
  return [inputs,lengths,labels]

train_dataloader=DataLoader(train_obj,batch_size=64,shuffle=True,collate_fn=custom_collate)
trainx,lenx,trainy=next(iter(train_dataloader))
print(f"Feature batch shape: {trainx.size()}")
print(f"Number of lengths {len(lenx)}")
print(f"Labels batch shape: {len(trainy)} {len(trainy[0])}")


val_dataloader=DataLoader(val_obj,batch_size=32,shuffle=True,collate_fn=custom_collate)
valx,lenx,valy=next(iter(val_dataloader))
print(f"Feature batch shape: {valx.size()}")
print(f"Number of lengths {len(lenx)}")
print(f"Labels batch shape: {len(valy)} {len(valy[0])}")



test_dataloader=DataLoader(test_obj,batch_size=32,shuffle=True,collate_fn=custom_collate)
testx,lenx,testy=next(iter(test_dataloader))
print(f"Feature batch shape: {testx.size()}")
print(f"Number of lengths {len(lenx)}")
print(f"Labels batch shape: {len(testy)} {len(testy[0])}")



Length of batch  64 2
len of inputs :  64 131
Feature batch shape: torch.Size([64, 579])
Number of lengths 64
Labels batch shape: 64 579
Length of batch  32 2
len of inputs :  32 400
Feature batch shape: torch.Size([32, 469])
Number of lengths 32
Labels batch shape: 32 469
Length of batch  32 2
len of inputs :  32 123
Feature batch shape: torch.Size([32, 814])
Number of lengths 32
Labels batch shape: 32 814


In [8]:
print(len(val_dataloader))

16


In [39]:
from torch import nn,tensor
import torch.optim as optim
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence

class lstm_model(nn.Module):
  def __init__(self,vocab_size,embed_dim,hidden_size,out_size):
    super(lstm_model, self).__init__()
    self.embedding_dim=embed_dim
    self.vocab_size=vocab_size
    self.output_size=out_size
    self.hidden_dim=hidden_size
    self.embeddings=nn.Embedding(vocab_size,embed_dim)
    self.bilstm=nn.LSTM(embed_dim,hidden_size,bidirectional=True,batch_first=True)
    self.fc=nn.Linear(2*hidden_size,out_size)
  def forward(self,inputs,lengths):
    print(f"Inputs: {inputs.shape}")
    embeds=self.embeddings(inputs)
    # batchsize * seqlen * embded_dim
    print(f"Embeddings shape : {embeds.shape}")

    if len(lengths)>1:
      packed_input = pack_padded_sequence(embeds, lengths, batch_first=True,enforce_sorted=False)

      # out,_=self.bilstm(embeds)
      packed_output, _ = self.bilstm(packed_input)
      output, _ = pad_packed_sequence(packed_output, batch_first=True)
    else:
      output,_=self.bilstm(embeds)
    print(f"LSTM {output.shape}")

    # batchsize * seqlen * (2. hidden dim)
    prob=self.fc(output)
    print(f"out shape : {prob.shape}")

    # batchsize * seqlen * out_size
    return prob

vocab_size=len(vocab)
embed_dim=120
hidden_dim=150
out_size=len(punct)
loss_fn = nn.CrossEntropyLoss()
epochs = 2
learning_rate = 0.1
model=lstm_model(vocab_size,embed_dim,hidden_dim,out_size)
optimizer = optim.Adam(model.parameters(), lr = learning_rate)
a=torch.nn.Softmax(dim=1)



In [ ]:
# training
from tqdm import tqdm
from sklearn.metrics import classification_report


e_num=0
target_names=[idx_to_punct[i] for i in range(len(punct))]
for epoch in range(epochs):
  train_loss = 0
  model.train()
  num_batch=0
  all_pred=[]
  all_actual=[]

  for batch in tqdm(train_dataloader, desc="Training"):
    x_train,lens, y_train = batch
    optimizer.zero_grad()# clears the gradients from prev iteration
    # print('train',x_train.shape)
    output = model(x_train,lens) # forward pass
    print(f"Output shape: {output.shape}")
    output=output.view(-1,output.shape[2])
    output = a(output)

    # print('test',output.shape,y_train.shape)
    loss = loss_fn(output, y_train.view(-1)) # calculate loss

    #backprop
    loss.backward() # compute gradients
    optimizer.step() # update parameters

    train_loss += loss.item()
    num_batch+=1

    val_loss = 0


  # Validation loop
  val_loss = 0
  model.eval()
  for i, (x_val,lens, y_val) in enumerate(val_dataloader):
    output = model(tensor(x_val),lens)
    output=output.view(-1,output.shape[2])
    output=a(output)
    all_pred+=[torch.argmax(output[i]) for i in range(len(output))]
    all_actual+=y_val.view(-1)
    loss=loss_fn(output,y_val.view(-1))
    # all_actual+=list(chain.from_iterable(y_val))
    # loss = loss_fn(output, tensor(list(chain.from_iterable(y_val))))
    val_loss += loss.item()
  print(classification_report(all_actual, all_pred, target_names=target_names))
  print(f'Epoch {e_num} \t\t Training Loss: {train_loss / len(train_dataloader)} \t\t Validation Loss: {val_loss / len(val_dataloader)}')
  e_num+=1
  print(train_loss/len(train_dataloader))
  print(f"Number of batches {num_batch}")




In [56]:
test_loss=0
all_actual=[]
all_pred=[]
for i, (x_test,lens, y_test) in enumerate(test_dataloader):
    output = model(tensor(x_test),lens)
    output=output.view(-1,output.shape[2])
    output=a(output)
    all_pred+=[torch.argmax(output[i]) for i in range(len(output))]
    all_actual+=y_test.view(-1)
    loss=loss_fn(output,y_test.view(-1))
    # all_actual+=list(chain.from_iterable(y_test))
    # loss = loss_fn(output, tensor(list(chain.from_iterable(y_test))))
    test_loss += loss.item()
print(classification_report(all_actual, all_pred, target_names=target_names))
print(f' Test Loss: {test_loss / len(test_dataloader)}')


Length of batch  32 2
len of inputs :  32 391
Inputs: torch.Size([32, 416])
Embeddings shape : torch.Size([32, 416, 120])


<ipython-input-56-2cc618d167c7>:5: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  output = model(tensor(x_test),lens)


LSTM torch.Size([32, 416, 300])
out shape : torch.Size([32, 416, 4])
Length of batch  32 2
len of inputs :  32 115
Inputs: torch.Size([32, 534])
Embeddings shape : torch.Size([32, 534, 120])
LSTM torch.Size([32, 534, 300])
out shape : torch.Size([32, 534, 4])
Length of batch  32 2
len of inputs :  32 242
Inputs: torch.Size([32, 477])
Embeddings shape : torch.Size([32, 477, 120])
LSTM torch.Size([32, 477, 300])
out shape : torch.Size([32, 477, 4])
Length of batch  32 2
len of inputs :  32 202
Inputs: torch.Size([32, 494])
Embeddings shape : torch.Size([32, 494, 120])
LSTM torch.Size([32, 494, 300])
out shape : torch.Size([32, 494, 4])
Length of batch  32 2
len of inputs :  32 239
Inputs: torch.Size([32, 1020])
Embeddings shape : torch.Size([32, 1020, 120])
LSTM torch.Size([32, 1020, 300])
out shape : torch.Size([32, 1020, 4])
Length of batch  32 2
len of inputs :  32 127
Inputs: torch.Size([32, 814])
Embeddings shape : torch.Size([32, 814, 120])
LSTM torch.Size([32, 814, 300])
out shape

In [53]:
# torch.save({
#             'model_state_dict': model.state_dict(),
#             'optimizer_state_dict': optimizer.state_dict(),
#             'embedding_dim': model.embedding_dim,
#             'hidden_dim': model.hidden_dim,
#             'vocab_size': vocab_size,
#             'vocab':vocab,
#             'punct':punct,
#             'idx_to_word':idx_to_word,
#             'idx_to_punct':idx_to_punct
#             }, 'pre_trained_sst.pth')
pre_trained=torch.load('./pre_trained_sst.pth')
print(pre_trained.keys())

dict_keys(['model_state_dict', 'optimizer_state_dict', 'embedding_dim', 'hidden_dim', 'vocab_size', 'vocab', 'punct', 'idx_to_word', 'idx_to_punct'])


In [54]:
import numpy as np
sentence='Hi How are you I am fine'
twords=nltk.word_tokenize(sentence)
words=[vocab[word] if word in vocab else vocab['OOV'] for word in twords]
words=torch.tensor([words])
lens=torch.tensor([len(words)])
out=model(words, lens)
out=a(out)
punctuations=[]
for probdist in out[0]:
  index=torch.argmax(probdist)
  punctuations.append(idx_to_punct[int(index)])
for i in range(len(twords)):
  print(twords[i],end=" ")
  if punctuations[i]!="None":
    print(f"{punctuations[i]} ",end="")
print("\n")
print(out.shape)

Inputs: torch.Size([1, 7])
Embeddings shape : torch.Size([1, 7, 120])
LSTM torch.Size([1, 7, 300])
out shape : torch.Size([1, 7, 4])
Hi . How . are , you I am . fine 

torch.Size([1, 7, 4])
